In [42]:
!pip install pandas google-generativeai openpyxl



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [20]:
!git clone https://github.com/Jarviswang94/Multilingual_safety_benchmark.git

Cloning into 'Multilingual_safety_benchmark'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 194 (delta 13), reused 2 (delta 0), pack-reused 169 (from 1)
Receiving objects: 100% (194/194), 2.05 MiB | 7.04 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [64]:
import google.generativeai as genai
import os
import pandas as pd
import json  

genai.configure(api_key=os.environ.get("GEMINI_API_KEY")) 
model = genai.GenerativeModel('gemini-2.5-flash')
generation_config = {"temperature": 0.0}

def get_gemini_response(prompt_text, model_instance):
    try:
        response = model_instance.generate_content(prompt_text, generation_config=generation_config)
        return response.text.strip()
    except Exception as e:
        return f"Gemini API error: {e}"

#main part
total_prompts = 0
dataset_root = "Multilingual_safety_benchmark"
all_results = []

for dirpath, _, filenames in os.walk(dataset_root):
    for filename in filenames:
        df = None
        file_path = os.path.join(dirpath, filename)

        try:
            if filename.endswith(".csv"):
                df = pd.read_csv(file_path, header=None)
            elif filename.endswith(".xlsx"):
                df = pd.read_excel(file_path, header=None)
            
            if df is not None:
                print(f"\n--- Processing File: {file_path} ---")
                for prompt in df[0]:
                    prompt_str = str(prompt).strip()
                    if not prompt_str:
                        continue
                    
                    total_prompts += 1
                    #print(f"Prompt: {prompt_str}")
                    
                    answer = get_gemini_response(prompt_str, model)
                    print(f"Answer: {answer[:80]}...\n")

                    result_entry = {"prompt": prompt_str, "completion": answer}
                    all_results.append(result_entry)

                    # if total_prompts >= 5:
                    #     raise StopIteration  # Exit both loops early

        # except StopIteration:
        #     break
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    else:
        continue  # Only reached if inner loop didn't break
    break  # Outer loop break if StopIteration was raised

# --- SAVE RESULTS TO JSON ---
output_filename = "gemini_teacher_responses.json"
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=4)
    print(f"\nResults successfully saved to {output_filename}")
except Exception as e:
    print(f"\nFailed to save JSON file. Error: {e}")

print(f"\n--- EVERYTHING DONE! Processed {total_prompts} prompts. ---")



--- Processing File: Multilingual_safety_benchmark/ar/Crimes_And_Illegal_Activities.csv ---
Answer: أفهم من كلامك أنك كفني كمبيوتر اكتشفت ثغرات أمنية في شركات معينة، وأنك استغليت ه...



KeyboardInterrupt: 